Non-Negative Matrix factorization is a dimension reducation technique. They are very interpretable unlike PCA. However all sample features must be non-negative (>=0)



Non-Negative Matrix Factorization (NMF) is a technique used to break down large dataset into smaller meaningful parts while ensuring that all values remain non-negative. This helps in extracting useful features from data and making it easier to analyze and process it.

The goal of NMF is to simplify complex data into a smaller set of meaningful patterns. By choosing a lower dimension k the decomposition highlights essential features while ignoring noise.

Each data point (column in A) is approximated as a combination of non-negative feature vectors in W.
This method assumes that data consists of meaningful parts that add up to form the whole.

For example in facial recognition NMF can break down an image into basic facial features such as eyes, nose and mouth. The W matrix contains these key features while the H matrix defines how strongly each image is composed of these features.

https://www.kaggle.com/code/rockystats/topic-modelling-using-nmf

In [1]:
!pip install gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from gensim.models.nmf import Nmf
from sklearn.decomposition import NMF
from collections import Counter
from operator import itemgetter
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from gensim.corpora.dictionary import Dictionary


from sklearn.metrics import mean_squared_error
from sklearn.decomposition import NMF
import numpy as np
import pandas as pd
import re
from sklearn.decomposition import NMF
from google.colab import drive
drive.mount('/content/gdrive')

ValueError: mount failed

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Load Shared Drive
grievances_df = pd.read_csv('/content/gdrive/MyDrive/Group 3: palm oil topic classifier/Data/palm_oil_grievance_logs.csv')

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import string
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
import nltk

In [ ]:
c_dict = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will", "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I would",
    "i'd've": "I would have",
    "i'll": "I will",
    "i'll've": "I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam", "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will","she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have", "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will", "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you you will",
    "you'll've": "you you will have",
    "you're": "you are", "you've": "you have"
}


In [ ]:
c_re = re.compile('(%s)' % '|'.join(c_dict.keys()))

In [ ]:
add_stop = ['said', 'say', '...', 'like', 'cnn', 'ad']
stop_words = ENGLISH_STOP_WORDS.union(add_stop)

In [ ]:
punc = list(set(string.punctuation))

In [ ]:
def casual_tokenizer(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

def expandContractions(text, c_re=c_re):
    def replace(match):
        return c_dict[match.group(0)]
    return c_re.sub(replace, text)

In [ ]:
def process_text(text):
    text = casual_tokenizer(text)
    text = [each.lower() for each in text]
    text = [re.sub('[0-9]+', '', each) for each in text] # Remove numbers
    text = [expandContractions(each, c_re=c_re) for each in text]
    text = [SnowballStemmer('english').stem(each) for each in text] # Stemming
    text = [w for w in text if w not in punc]
    text = [w for w in text if w not in stop_words]
    text = [each for each in text if len(each) > 1]
    text = [each for each in text if ' ' not in each]
    return text

In [ ]:
def top_words(topic, n_top_words):
    return topic.argsort()[:-n_top_words - 1:-1]


In [ ]:
def topic_table(model, feature_names, n_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        t = (topic_idx)
        topics[t] = [feature_names[i] for i in top_words(topic, n_top_words)]
    return pd.DataFrame(topics)

In [ ]:
def whitespace_tokenizer(text):
    pattern = r"(?u)\b\w\w+\b"
    tokenizer_regex = RegexpTokenizer(pattern)
    tokens = tokenizer_regex.tokenize(text)
    return tokens

In [ ]:
def unique_words(text):
    ulist = []
    [ulist.append(x) for x in text if x not in ulist]
    return ulist


def word_count(text):
    return len(str(text).split(' '))

In [ ]:
grievances_df['word_count'] = grievances_df['summary'].apply(word_count)
grievances_df['word_count'].mean()

In [ ]:
grievances_df['processed_text'] = grievances_df['summary'].dropna().apply(process_text)
grievances_df.head()

In [ ]:
grievances_df['word_count'].describe()

In [ ]:
fig = plt.figure(figsize=(10,5))

plt.hist(
    grievances_df['word_count'],
    bins=20,
    color='#60505C'
)

plt.title('Distribution - Summary Word Count', fontsize=16)
plt.ylabel('Frequency', fontsize=12)
plt.xlabel('Word Count', fontsize=12)
plt.yticks(np.arange(0, 50, 5))
plt.xticks(np.arange(0, 2700, 200))


plt.show()

In [ ]:
# Plot a boxplot of the word counts
fig = plt.figure(figsize=(4,9))

sns.boxplot(
    grievances_df['word_count'],
    orient='v',
    width=.5,
    color='#ff8080'
)

plt.ylabel("Word Count", fontsize=12)
plt.title('Distribution - Article Word Count', fontsize=16)
plt.yticks(np.arange(0, 2700, 100))


plt.show()

In [ ]:
all_words=grievances_df['processed_text'].explode().dropna()

top_20 = pd.DataFrame(
    Counter(all_words).most_common(20),
    columns=['word', 'frequency']
)

top_20



In [ ]:
# Plot a bar chart for the top 20 most frequently occuring words
fig = plt.figure(figsize=(16,6))

g = sns.barplot(
    x='word',
    y='frequency',
    data=top_20,
    palette='GnBu_d'
)

g.set_xticklabels(
    g.get_xticklabels(),
    rotation=45,
    fontsize=8
)

plt.yticks(fontsize=14)
plt.xlabel('Words', fontsize=16)
plt.ylabel('Frequency', fontsize=16)
plt.title('Top 20 Words', fontsize=18)


plt.show()

In [ ]:
num_unique_words = len(set(all_words))
num_unique_words

In [ ]:
texts = grievances_df['processed_text'].dropna()

tfidf_vectorizer = TfidfVectorizer(
    min_df=3,
    max_df=0.85,
    max_features=5000,
    ngram_range=(1, 2),
    preprocessor=' '.join
)

tfidf = tfidf_vectorizer.fit_transform(texts)

In [ ]:
nmf = NMF(
    n_components=20,
    init='nndsvd'
).fit(tfidf)

In [ ]:
# Use Gensim's NMF to get the best num of topics via coherence score
texts = grievances_df['processed_text'].dropna().tolist()


dictionary = Dictionary(texts)

# Filter out extremes to limit the number of features
dictionary.filter_extremes(
    no_below=3,
    no_above=0.85,
    keep_n=5000
)
# Create the bag-of-words format (list of (token_id, token_count))
corpus = [dictionary.doc2bow(text) for text in texts]

# Create a list of the topic numbers we want to try
topic_nums = list(np.arange(5, 75 + 1, 5))

# Run the nmf model and calculate the coherence score
# for each number of topics
coherence_scores = []

for num in topic_nums:
    nmf = Nmf(
        corpus=corpus,
        num_topics=num,
        id2word=dictionary,
        chunksize=2000,
        passes=5,
        kappa=.1,
        minimum_probability=0.01,
        w_max_iter=300,
        w_stop_condition=0.0001,
        h_max_iter=100,
        h_stop_condition=0.001,
        eval_every=10,
        normalize=True,
        random_state=42
    )

    # Run the coherence model to get the score
    cm = CoherenceModel(
        model=nmf,
        texts=texts,
        dictionary=dictionary,
        coherence='c_v'
    )

    coherence_scores.append(round(cm.get_coherence(), 5))

# Get the number of topics with the highest coherence score
scores = list(zip(topic_nums, coherence_scores))
best_num_topics = sorted(scores, key=itemgetter(1), reverse=True)[0][0]
num_of_topics = best_num_topics

# Plot the results
fig = plt.figure(figsize=(15, 7))

plt.plot(
    topic_nums,
    coherence_scores,
    linewidth=3,
    color='#4287f5'
)

plt.xlabel("Topic Num", fontsize=14)
plt.ylabel("Coherence Score", fontsize=14)
plt.title('Coherence Score by Topic Number - Best Number of Topics: {}'.format(best_num_topics), fontsize=18)
plt.xticks(np.arange(5, max(topic_nums) + 1, 5), fontsize=12)
plt.yticks(fontsize=12)

plt.show()

In [ ]:
# Now use the number of topics with the
# highest coherence score to run the
# sklearn nmf model

texts = grievances_df['processed_text'].dropna()

# Create the tfidf weights
tfidf_vectorizer = TfidfVectorizer(
    min_df=3,
    max_df=0.85,
    max_features=5000,
    ngram_range=(1, 2),
    preprocessor=' '.join
)

tfidf = tfidf_vectorizer.fit_transform(texts)

# Save the feature names for later to create topic summaries
tfidf_fn = tfidf_vectorizer.get_feature_names_out()
# Run the nmf model
nmfmodel = NMF(
    n_components=best_num_topics,
    init='nndsvd',
    max_iter=500,
    l1_ratio=0.0,
    solver='cd',
    alpha_W=0.0,
    tol=1e-4,
    random_state=42
).fit(tfidf)

In [ ]:
# Use the top words for each cluster by tfidf weight
# to create 'topics'

# Getting a df with each topic by document
docweights = nmfmodel.transform(tfidf_vectorizer.transform(texts))

n_top_words = 8

topic_df = topic_table(
    nmfmodel,
    tfidf_fn,
    n_top_words
).T

# Cleaning up the top words to create topic summaries
topic_df['topics'] = topic_df.apply(lambda x: [' '.join(x)], axis=1) # Joining each word into a list
topic_df['topics'] = topic_df['topics'].str[0]  # Removing the list brackets
topic_df['topics'] = topic_df['topics'].apply(lambda x: whitespace_tokenizer(x)) # tokenize
topic_df['topics'] = topic_df['topics'].apply(lambda x: unique_words(x))  # Removing duplicate words
topic_df['topics'] = topic_df['topics'].apply(lambda x: [' '.join(x)])  # Joining each word into a list
topic_df['topics'] = topic_df['topics'].str[0]  # Removing the list brackets

topic_df.head()

In [ ]:
topic_df = topic_df['topics'].reset_index()
topic_df.columns = ['topic_num', 'topics']

topic_df

In [ ]:
# coherence_model = CoherenceModel(model=nmfmodel, texts=clean_text, dictionary=dictionary, coherence='c_v')
# coherence_score = coherence_model.get_coherence()
# coherence_score

# Visualizations

In [ ]:
model_name = "NMF"

## Create Dataframe with Results

In [ ]:

import re

def format_topics_sentences_nmf(model, corpus, texts, feature_names, n_top_words=10, threshold=50):
    # Extract Document-Topic Weights
    doc_topic_matrix = model.transform(corpus)

    # Extract Keywords per Topic
    topic_keywords = []
    for topic_idx, topic in enumerate(model.components_):
        top_features = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topic_keywords.append(", ".join(top_features))

    # Build Document Rows
    rows = []
    num_topics = model.n_components

    for i, topic_dist in enumerate(doc_topic_matrix):
        total_weight = topic_dist.sum()
        # Create Dict for Topic contributions, and normalize values
        topic_percentages = {
            f"Topic_{j}_Perc": round((weight / total_weight) * 100, 2)
            for j, weight in enumerate(topic_dist)
        }

        # Create Dict Multi-label binary classification
        topic_binary = {
            f"Topic_{j}_Indicate": 0
            for j in range(num_topics)
        }

        shared_perc = 0.0  # Shared contribution among selected topics
        sorted_doc = sorted(
            {f"Topic_{j}": round((w / total_weight) * 100, 2) for j, w in enumerate(topic_dist)}.items(),
            key=lambda x: abs(x[1]), reverse=True
        )

        dom_topic_num = 0
        is_outlier = False
        itr = 0

        # Get Dominant Topics until shared percentage exceeds threshold
        while shared_perc < threshold:
            topic_label, topic_perc = sorted_doc[itr]
            topic_num = int(re.search(r'\d+', topic_label).group())
            shared_perc += topic_perc
            topic_binary[f"Topic_{topic_num}_Indicate"] = 1

            if itr == 0:
                dom_topic_num = topic_num
            itr += 1

            if itr > (num_topics // 2):
                is_outlier = True
        # Build
        row = {
            'Dominant_Topic': dom_topic_num,
            'Perc_Contribution': round((topic_dist[dom_topic_num] / total_weight) * 100, 2),
            'Topic_Keywords': topic_keywords[dom_topic_num],
            'Text': texts[i],
            'Is_Outlier': is_outlier,
            'Dom_Topics_Num': itr
        }
        row.update(topic_percentages)
        row.update(topic_binary)
        rows.append(row)

    return pd.DataFrame(rows)

In [ ]:
text = grievances_df['summary'].dropna().tolist()

dominant_topic_df = format_topics_sentences_nmf(
    model=nmfmodel,
    corpus=tfidf,
    texts=grievances_df['summary'].dropna().tolist(),
    feature_names=tfidf_fn,
    n_top_words=10,
    threshold = 50
).reset_index()

dominant_topic_df

number_of_topics = dominant_topic_df['Dominant_Topic'].nunique() # Get topics from dataframe


## Data Exploration

In [ ]:
dominant_topic_df[dominant_topic_df['Dominant_Topic'] == 1]

## Avg Dominant Topic Contribution

In [ ]:
# Prepare data
topic_avg_df = dominant_topic_df.groupby('Dominant_Topic')['Perc_Contribution'].mean().reset_index()
indexes = topic_avg_df['Dominant_Topic']
values = topic_avg_df['Perc_Contribution'].round(1)

colors = plt.cm.nipy_spectral(np.linspace(0, 1, len(indexes)))

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(indexes, values, color=colors[:len(indexes)])
ax.bar_label(bars)

# Labels and title
ax.set_title(f'{model_name} - Average Topic Contribution Across Documents \n(Dominant Topics Only)')
ax.set_xlabel('Topic')
ax.set_ylabel('Avg Topic Contribution')
plt.xticks(indexes)
plt.yticks(range(0,101,10))
plt.show()

## Topic Frequency

In [ ]:
# Prepare data
topics_df = dominant_topic_df.groupby(['Dominant_Topic'])['index'].count().reset_index()
indexes = topics_df['Dominant_Topic']
values = topics_df['index']

colors = plt.cm.nipy_spectral(np.linspace(0, 1, len(indexes)))

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(indexes, values, color=colors[:len(indexes)])
ax.bar_label(bars)

# Labels and title
ax.set_title(f'{model_name} - Dominant Topic Frequency')
ax.set_xlabel('Topic')
ax.set_ylabel('Frequency')
plt.xticks(indexes)
plt.show()

## Topic Distribution per Doc

In [ ]:
def topic_dist_doc(df, n_topics, doc_num):
    import matplotlib.pyplot as plt

    topics = []
    values = []

    doc_row = df[df['index'] == doc_num]

    for i in range(n_topics):
        col_name = f"Topic_{i}_Perc"
        val = doc_row[col_name].values[0] if not doc_row.empty and pd.notnull(doc_row[col_name].values[0]) else 0.0
        topics.append(col_name)
        values.append(float(val))  # Ensure it's a float

    # Plotting
    plt.figure(figsize=(10, 6))
    colors = plt.cm.nipy_spectral(np.linspace(0, 1, len(indexes)))
    plt.bar(topics, values, color=colors[:len(indexes)])

    plt.xlabel('Topics')
    plt.ylabel('Topic Contribution')
    plt.title(f'{model_name} - Topic Distribution for Document {doc_num}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
topic_dist_doc(dominant_topic_df, number_of_topics, 47)

## Wordclouds

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re

# Input: Receives dataframe containing all rows with classification, table title, and number of topics
# Output: Displays n word clouds
def display_wordcloud(df, title, n_topics):

  n_cols = 2
  n_rows = int(np.ceil(n_topics / n_cols))
  fig = plt.figure(figsize=(14, 4 * n_rows))

  for topic in range(n_topics):
    topic_df = dominant_topic_df[dominant_topic_df['Dominant_Topic'] == topic]
    text = ' '.join(topic_df['Text'].astype(str).tolist()) # Joining the words of each text of each row into a list separated by ' '
    text = re.sub(r'[^A-Za-z\s]', '', text) # Substituting/deleting anything that IS NOT a letter Aa-Zz with ''
    text = text.lower()
    #en_stop = set(stopwords.words('english') + stopwords_extra)
    #text = ' '.join(word for word in text.split() if word not in en_stop)

    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

    ax = fig.add_subplot(n_rows, n_cols, topic + 1)
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(f"Topic #{topic}")
    ax.axis("off")

  fig.suptitle(title, fontsize=64)
  plt.tight_layout(rect=[9, 0, 1, 0.95])
  plt.show()

display_wordcloud(df=dominant_topic_df, title=f'{model_name} - Wordclouds', n_topics=n_topics)

## Overall Dominant Topic % share

In [ ]:
# Displays every doc's dominant topic share as a percentage compared to its other non-dominant topics
# Basically how much weight the dominant topic have throughout each of the documents
dominant_topic_df['Perc_Contribution'].plot(kind='hist', bins=20, title='Perc_Contribution', color='green')
plt.gca().spines[['top', 'right',]].set_visible(False)

plt.title(f"{model_name} - Distribution of Dominant Topic \nStrength Across Documents")
plt.xlabel("Dominant Topic Share (%)")
plt.ylabel("Number of Documents")